# KNN Classification-based Prediction of Heart Disease Diagnoses

## Introduction
Heart disease is a significant global health concern; it is the *leading cause of death globally*. To address this issue effectively, it is vital to develop accurate diagnostic tools. Fortunately, there are many patient attributes that could serve as indicators or possible warning signs for heart disease, which could accelerate the process of diagnosis and treatment. In this project, we will employ classification techniques to predict the diagnosis of heart disease based on a few of these attributes.

**Research question:** Will a patient be diagnosed with heart disease, given their age, resting blood pressure, cholesterol levels, and maximum heart rate achieved, while considering sex differences?

The dataset we will use to answer this question is the UCI Heart Disease dataset, which comprises multivariate numerical and categorical data on 14 patient attributes. These include age, sex, chest pain type, etc. along with the patient's diagnosis. For this project, we will focus on a subset of attributes: 
- age
- resting blood pressure
- serum cholesterol
- maximum heart rate achieved

to predict the diagnosis of heart disease. We will filter the dataset to include only records from the Cleveland location to ensure consistency and relevance. By the end of this project, we aim to develop accurate predictive models for diagnosing heart disease based on patient attributes.

## Exploratory data analysis

In [ ]:
# Run this cell before continuing
library(tidyverse)
library(repr)
library(tidymodels)
library(RColorBrewer)
options(repr.matrix.max.rows = 5)
set.seed(69)

In [ ]:
heart <- read_csv("https://raw.githubusercontent.com/chiefpat450119/dsci-project/main/data/heart_disease_uci.csv")

We then filter the data to look at **only one location (Cleveland)** to reduce the effect of possible confounding variables introduced by the local environment.

In [ ]:
cleveland_heart <- heart |>
filter(dataset == "Cleveland")

In order to use classification, we need to change the predicted variable (num) to a *factor* with more understandable levels. From the information presented in the dataset, num = 0 represents absence of disease, while num = 1, 2, 3, 4 represent stages of heart disease. The refactoring will be done based on the following interpretation:
- 0 = "absent"
- 1 = "mild"
- 2 = "moderate"
- 3 = "severe"
- 4 = "critical"

In [ ]:
heart_refactored <- cleveland_heart |>
mutate(disease = as_factor(num)) |>
mutate(disease = fct_recode(disease, "absent" = "0", "mild" = "1", "moderate" = "2", "severe" = "3", "critical" = "4"))
heart_refactored

The data is already in a tidy format. We now split the data into a training and a testing set, making sure there are equal ratios of each disease classification in both sets. Let's take a look at how many cases are in each disease class, and then break it down by sex.

In [ ]:
heart_split <- initial_split(heart_refactored, prop = 0.75, strata = disease)  
heart_train <- training(heart_split)   
heart_test <- testing(heart_split)

In [ ]:
heart_train_summary <- heart_train |>
group_by(disease) |>
summarize(count = n())
heart_train_summary

In [ ]:
options(repr.matrix.max.rows = 10)
heart_train_split <- heart_train |>
group_by(disease, sex) |>
summarize(count = n())
heart_train_split

There is a difference in the distribution of cases between male and female patients. There is a **significantly greater proportion** of patients in the disease classes for male patients. This will need to be kept in mind for our analysis. 

Some disease classes have **very few observations,** especially for female patients. This will make it hard to train a useful classification model. Thus, let's **combine all the disease classes into one**:

In [ ]:
options(repr.matrix.max.rows = 5)
heart_train <- mutate(heart_train, disease = case_when(disease == "absent" ~ "absent", TRUE ~ "present"))
heart_test <- mutate(heart_test, disease = case_when(disease == "absent" ~ "absent", TRUE ~ "present"))
heart_train

Let's see how many missing values there are for each predictor.

In [ ]:
missing <- enframe(colSums(is.na(heart_train))) |>
pivot_wider(names_from = "name", values_from = "value")
missing

### Selected predictors
- Age
- Resting blood pressure (trestbps)
- Serum cholesterol (chol)
- Maximum heart rate achieved (thalch)

In [ ]:
# Let's look at how two of predictor variables correlate with heart disease.
heart_age_bp_plot <- heart_train |>
  ggplot(aes(x = age, y = trestbps, color = disease)) +
  geom_point(alpha = 0.6) + 
  labs(x = "Age (years)", y = "Resting Blood Pressure (mmHg)", color = "Disease Diagnosis") +
  ggtitle("Figure 1: Relationship between Resting Blood Pressure & Age") +
  theme(
    plot.title = element_text(size = 16),  # Adjust title size
    axis.title = element_text(size = 14),  # Adjust axis label size
    legend.title = element_text(size = 14) # Adjust legend title size
  ) +
  facet_grid(cols = (vars(sex)))

heart_age_bp_plot

In [ ]:
# Now the other two predictor variables
heart_chol_hr_plot <- ggplot(heart_train, aes(x = chol, y = thalch, color = disease)) + 
  geom_point(alpha = 0.6) + 
  labs(x = "Cholesterol Level (mg/dL)",  # Including unit
       y = "Maximum Heart Rate (bpm)", 
       color = "Disease diagnosis") + 
  ggtitle("Relationship between Maximum Heart Rate & Cholesterol Level") +
  theme(
    plot.title = element_text(size = 16),  # Adjust title size
    axis.title = element_text(size = 14),  # Adjust axis label size
    legend.title = element_text(size = 14) # Adjust legend title size
  ) +
  facet_grid(cols = (vars(sex)))
heart_chol_hr_plot

# Methods
To build a model for predicting the diagnosis of heart disease, we will follow the outlined approach.

In the data preprocessing phase, we will focus on filtering the dataset to include only observations from the Cleveland area, aiming to reduce potential confounding variables introduced by regional differences. Then, we will select the following predictors: age, resting blood pressure, cholesterol, and maximum heart rate.  
These were chosen because they are numerical variables, which are much more useful than the other categorical variables for KNN classification, and they also do not have any missing data. The disease classes will be combined into one because some of the original classes had too few observations to use classification effectively.

For model fitting and evaluation, we will split the dataset into training and testing sets, separately for male and female, due to the notable sex differences in diagnoses and predictor variables (see graphs above). We will be training a K-nearest neighbours classification model. The model will be trained on the training set, using cross-validation to choose the optimal value of K, then evaluated on the testing set using accuracy, precision, and recall. Recall will be prioritized because false negative predictions (classifying disease "present" as "absent") are more costly. This evaluation process will be conducted separately for male and female categories to account for sex-specific variations.

We will visualize our findings by creating four scatter plots with the **training data**, each representing a unique combination of variables. For each sex, we will plot blood pressure against age and maximum heart rate against cholesterol, colouring points by the patient's predicted diagnosis. The plot background will be coloured according to the predicted class at each point. This can be done for several values of K during the cross-validation to assess under- or over-fitting. These scatter plots will serve to identify any discernible patterns or correlations between the selected variables and heart disease diagnosis, facilitating the interpretation of our predictive models.  

We will also visualize the accuracy, precision, and recall when the model is evaluated on the **test data** using bar charts, along with bar charts visualizing the confusion matrix (true positives, false positives, etc.) for each sex.

### Criteria for Methods and Results
- describe in written English the methods you used to perform your analysis from beginning to end that narrates the code the does the analysis.
- your report should include code which
- 
loads data from the original source on the eb
- 
wrangles and cleans the data from it's original (downloaded) format to the format necessary for the planned anasis
- s
performs a summary of the data set that is relevant for exploratory data analysis related to the planned aysis
- s 
creates a visualization of the dataset that is relevant for exploratory data analysis related to the plannedlysis
- sis
performs the daalysis
- ysis
creates a visualization ofnae a
-
- ysis 
note: all tables and figure should have a figure/table number and a legend

## Expected outcomes and significance
Based on the outlined methods, we expect to find predictive models capable of accurately diagnosing heart disease based on patient attributes like age, blood pressure, cholesterol, and maximum heart rate achieved. Such findings could lead to improved early detection and treatment, potentially reducing the burden of heart disease globally. Future questions may explore the effectiveness of these models across different demographics, as well as their integration into clinical practice for better patient outcomes.

## Data Analysis

In [ ]:
# Split into training and testing for both male and female
# Perform cross-validation to choose K for both male and female
# Analyse performance (precision and recall?) Decide which is most important
# Visualise predictions
# Predict on the testing set using the optimal value of K for both male and female
# Visualise metrics and results on the test data

## Discussion
- summarize what you found
- discuss whether this is what you expected to find
- 
discuss what impact could such findings hae?
- 
discuss what future questions could this lead to?

## References
- At least 2 citations of literature relevant to the project (format is your choice, just be consistent across the references).
- 
Make sure to cite the source of your data as well.